In [1]:
import sqlite3
import pandas as pd

# Dictionary with dataset URLs and column names
url_dict = {
    'airports': ('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat', 
                 ["airport_id", "name", "city", "country", "iata", "icao", "latitude", "longitude", "altitude",
                  "timezone", "dst", "tz_database_timezone", "type", "source"]),
    'airlines': ('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat',
                 ['airline_id', 'name', 'alias', 'iata', 'icao', 'callsign', 'country', 'active']),
    'routes': ('https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat',
               ['airline', 'airline_id', 'source_airport', 'source_airport_id', 'destination_airport',
                'destination_airport_id', 'codeshare', 'stops', 'equipment']),
    'planes': ('https://raw.githubusercontent.com/jpatokal/openflights/master/data/planes.dat',
               ['name', 'iata_code', 'icao_code'])
}

# Connect to SQLite database
conn = sqlite3.connect("openflights.db")
cursor = conn.cursor()

In [2]:
# Load data into SQLite
for db_name, (url, columns) in url_dict.items():
    df = pd.read_csv(url, names=columns)
    df.columns = df.columns.str.lower().str.replace(" ", "_")  # Convert column names to lowercase and remove spaces
    df.to_sql(db_name, conn, if_exists='replace', index=False)

# Function to execute queries and return results
def execute_query(query):
    return pd.read_sql_query(query, conn)

In [3]:
# 1. Retrieve all columns for the first 5 rows from the `airports` table.
query_1 = """
SELECT * FROM airports LIMIT 5;
"""
df1 = execute_query(query_1)
print("1. First 5 rows from airports table:\n", df1)

# 2. Select name, city, and country for airports in the United States.
query_2 = """
SELECT name, city, country FROM airports WHERE country = 'United States';
"""
df2 = execute_query(query_2)
print("\n2. Airports in the United States:\n", df2)

# 3. Find active airlines with at least 3 distinct source airports.
query_3 = """
SELECT a.name, a.airline_id, COUNT(DISTINCT r.source_airport_id) AS source_airport_count
FROM airlines a
JOIN routes r ON a.airline_id = r.airline_id
WHERE a.active = 'Y'
GROUP BY a.name, a.airline_id
HAVING COUNT(DISTINCT r.source_airport_id) >= 3
ORDER BY source_airport_count DESC;
"""
df3 = execute_query(query_3)
print("\n3. Active airlines with at least 3 distinct source airports:\n", df3)

# 4. List airports that serve as a source airport along with details and total departing routes.
query_4 = """
SELECT a.name, a.city, a.country, a.altitude, COUNT(r.source_airport_id) AS total_routes
FROM airports a
JOIN routes r ON a.airport_id = r.source_airport_id
GROUP BY a.name, a.city, a.country, a.altitude
HAVING COUNT(r.source_airport_id) > 0
ORDER BY total_routes DESC, a.altitude DESC;
"""
df4 = execute_query(query_4)
print("\n4. Airports with at least one departing route:\n", df4)

# 5. Rank source airports based on route count, only include those with at least 5 routes.
query_5 = """
SELECT source_airport_id, COUNT(*) AS route_count, 
       RANK() OVER (ORDER BY COUNT(*) DESC) AS rank
FROM routes
GROUP BY source_airport_id
HAVING COUNT(*) >= 5;
"""
df5 = execute_query(query_5)
print("\n5. Ranked source airports with at least 5 routes:\n", df5)

1. First 5 rows from airports table:
    airport_id                                         name          city  \
0           1                               Goroka Airport        Goroka   
1           2                               Madang Airport        Madang   
2           3                 Mount Hagen Kagamuga Airport   Mount Hagen   
3           4                               Nadzab Airport        Nadzab   
4           5  Port Moresby Jacksons International Airport  Port Moresby   

            country iata  icao  latitude   longitude  altitude timezone dst  \
0  Papua New Guinea  GKA  AYGA -6.081690  145.391998      5282       10   U   
1  Papua New Guinea  MAG  AYMD -5.207080  145.789001        20       10   U   
2  Papua New Guinea  HGU  AYMH -5.826790  144.296005      5388       10   U   
3  Papua New Guinea  LAE  AYNZ -6.569803  146.725977       239       10   U   
4  Papua New Guinea  POM  AYPY -9.443380  147.220001       146       10   U   

   tz_database_timezone     ty

In [4]:
# 6. List routes where airline, source, and destination airports are in the same country.
query_6 = """
SELECT al.name AS airline_name, al.country AS airline_country, 
       sa.name AS source_airport, sa.city AS source_city, 
       da.name AS destination_airport, da.city AS destination_city
FROM routes r
JOIN airlines al ON r.airline_id = al.airline_id
JOIN airports sa ON r.source_airport_id = sa.airport_id
JOIN airports da ON r.destination_airport_id = da.airport_id
WHERE al.country = sa.country AND al.country = da.country;
"""
df6 = execute_query(query_6)
print("\n6. Routes where airline, source, and destination are in the same country:\n", df6)

# 7. Find airports not appearing in the routes table (neither as source nor destination).
query_7 = """
SELECT name, city, country
FROM airports
WHERE airport_id NOT IN (SELECT source_airport_id FROM routes)
AND airport_id NOT IN (SELECT destination_airport_id FROM routes);
"""
df7 = execute_query(query_7)
print("\n7. Airports not appearing in the routes table:\n", df7)

# 8. List each source airport with a comma-separated list of distinct destination airports.
query_8 = """
SELECT source_airport_id, 
       GROUP_CONCAT(destination_airport_id, ', ') AS destinations
FROM (
    SELECT DISTINCT source_airport_id, destination_airport_id
    FROM routes
) AS distinct_routes
GROUP BY source_airport_id
HAVING COUNT(destination_airport_id) > 1;
"""
df8 = execute_query(query_8)
print("\n8. Source airports with multiple distinct destinations:\n", df8)

# 9. Update the `active` status of a specific airline_id (e.g., 1234).
query_9 = """
UPDATE airlines
SET active = 'N'
WHERE airline_id = 1234;
"""
cursor.execute(query_9)
conn.commit()
print("\n9. Updated active status for airline with ID 1234.")

# 10. Transaction to delete routes where stops > 0 and verify.
try:
    cursor.execute("BEGIN TRANSACTION;")
    cursor.execute("DELETE FROM routes WHERE stops > 0;")
    conn.commit()
    print("\n10. Deleted all routes where stops > 0.")

    query_10 = "SELECT * FROM routes WHERE stops > 0;"
    df10 = execute_query(query_10)
    print("\n10. Verification - Routes with stops > 0:\n", df10)

except Exception as e:
    conn.rollback()
    print("\n10. Transaction failed. Rolling back changes.", e)

# Close the connection
conn.close()


6. Routes where airline, source, and destination are in the same country:
            airline_name airline_country  \
0        Star Peru (2I)            Peru   
1        Star Peru (2I)            Peru   
2        Star Peru (2I)            Peru   
3        Star Peru (2I)            Peru   
4        Star Peru (2I)            Peru   
...                 ...             ...   
25293  Regional Express       Australia   
25294  Regional Express       Australia   
25295  Regional Express       Australia   
25296  Regional Express       Australia   
25297  Regional Express       Australia   

                                          source_airport  source_city  \
0            Coronel FAP Alfredo Mendivil Duarte Airport     Ayacucho   
1         Alejandro Velasco Astete International Airport        Cuzco   
2         Alejandro Velasco Astete International Airport        Cuzco   
3          Alferez Fap David Figueroa Fernandini Airport      Huánuco   
4      Coronel FAP Francisco Secada Vignet